This notebook will be mainly used for the capstone project

In [2]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

## Creating the dataframe

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#Scraping the table from wiki and placing it in a pandas dataframe
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
Postcode = df["Postcode"].tolist()
Borough = df["Borough"].tolist()
Neighbourhood = df["Neighbourhood"].tolist()
df = [Postcode, Borough, Neighbourhood]
df = pd.DataFrame(df)
df = df.transpose()
df.rename(columns = {0: "Postalcode", 1:"Borough", 2: "Neighbourhood"}, inplace = True)

#Loop to rename all Neighborhoods with "Not assigned" to Borough Names
i = 0
for n in df["Neighbourhood"]:
    if n == "Not assigned":
        temp = df["Borough"][i]
        df["Neighbourhood"][i] = temp
    i = i + 1

#Removing all "Not assigned" boroughs
df = df[df.Borough != 'Not assigned']

#Combining rows with the same postcode
df = df.groupby(by=['Postalcode','Borough']).agg(lambda x: ', '.join(x))
df.reset_index(level=['Postalcode','Borough'], inplace=True)


In [5]:
df.shape

(103, 3)

## Getting the latitude and longitude

In [6]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


!conda install -c conda-forge geocoder --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [7]:
import geocoder # import geocoder
latitude = []
longitude = []
postalcode_1 = []
for n in df["Postalcode"]:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(n))
        lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1]) 
    postalcode_1.append(n)

In [8]:
B = df["Borough"].tolist()
N = df["Neighbourhood"].tolist()

df_latlong = pd.DataFrame([postalcode_1, B, N, latitude, longitude])
df_latlong = df_latlong.transpose()
df_latlong.rename(columns = {0: "Postalcode", 1:"Borough", 2: "Neighbourhood", 3: "Latitude", 4: "Longitude"}, inplace = True)


In [9]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
tor_latitude = location.latitude
tor_longitude = location.longitude

In [10]:
import folium 
map_Toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_latlong['Latitude'], df_latlong['Longitude'], df_latlong['Borough'], df_latlong['Neighbourhood']):
    label = '{}, {}'.format(df_latlong['Neighbourhood'], df_latlong['Borough'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## Analysis of Boroughs that only contain the word "Toronto" 

In [39]:
#Creating a new dataframe with Boroughs that only contain the word "Toronto"
df_simp = df_latlong[df_latlong['Borough'].str.contains("Toronto")]

CLIENT_ID = 'HRLJQDSL22E520PIRUKGAWSVOLJ0YUW51QF0STZLQHANL54E' # your Foursquare ID
CLIENT_SECRET = '5ZCV0JXUZWAQYOQDNZ3KVJXZVJSS2VQ1UEUOOSFZYAGOKYLP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
Toronto_venues = getNearbyVenues(names=df_simp['Borough'],
                                   latitudes=df_simp['Latitude'],
                                   longitudes=df_simp['Longitude']
                                  )


East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
East Toronto


In [42]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1748, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
1,East Toronto,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,East Toronto,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
3,East Toronto,43.676845,-79.295225,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,East Toronto,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood


In [43]:
Toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,82,82,82,82,82,82
Downtown Toronto,1243,1243,1243,1243,1243,1243
East Toronto,183,183,183,183,183,183
West Toronto,240,240,240,240,240,240


In [44]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 210 uniques categories.


In [45]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Borough column back to dataframe
Toronto_onehot['Borough'] = Toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Borough,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
Toronto_onehot.shape

(1748, 211)

In [47]:
Toronto_grouped = Toronto_onehot.groupby('Borough').mean().reset_index()
Toronto_grouped

,Borough,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Central Toronto,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,0.012195,...,0.000000,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000
1,Downtown Toronto,0.000805,0.016895,0.000000,0.007241,0.000805,0.001609,0.008045,0.000805,0.002414,...,0.000805,0.002414,0.000000,0.001609,0.012872,0.003218,0.005632,0.006436,0.001609,0.002414
2,East Toronto,0.000000,0.027322,0.005464,0.000000,0.000000,0.010929,0.010929,0.000000,0.005464,...,0.000000,0.000000,0.005464,0.000000,0.010929,0.000000,0.005464,0.000000,0.000000,0.000000
3,West Toronto,0.000000,0.012500,0.000000,0.008333,0.000000,0.004167,0.012500,0.000000,0.004167,...,0.000000,0.000000,0.004167,0.000000,0.016667,0.000000,0.016667,0.004167,0.000000,0.004167


In [48]:
Toronto_grouped.shape

(4, 211)

In [31]:
num_top_venues = 5

for hood in Toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06
2  Sandwich Place  0.06
3            Park  0.05
4     Pizza Place  0.05


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.05
2          Restaurant  0.04
3  Italian Restaurant  0.03
4               Hotel  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1         Pizza Place  0.04
2                Café  0.04
3  Italian Restaurant  0.04
4                 Pub  0.03


----West Toronto----
         venue  freq
0  Coffee Shop  0.08
1       Bakery  0.04
2   Restaurant  0.04
3         Café  0.04
4          Bar  0.03




In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = Toronto_grouped['Borough']

for ind in np.arange(Toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Italian Restaurant,Restaurant,Gym,Health & Beauty Service
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Japanese Restaurant,American Restaurant,Gastropub
2,East Toronto,Coffee Shop,Pizza Place,Italian Restaurant,Café,Bar,Sushi Restaurant,Pub,Park,Steakhouse,American Restaurant
3,West Toronto,Coffee Shop,Café,Restaurant,Bakery,Bar,Pizza Place,Sandwich Place,Park,Italian Restaurant,Gym


## Clustering Boroughs

In [50]:
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = Toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1], dtype=int32)

In [51]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_simp

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

Toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6768,-79.2952,2,Coffee Shop,Pizza Place,Italian Restaurant,Café,Bar,Sushi Restaurant,Pub,Park,Steakhouse,American Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6833,-79.3551,2,Coffee Shop,Pizza Place,Italian Restaurant,Café,Bar,Sushi Restaurant,Pub,Park,Steakhouse,American Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668,-79.3147,2,Coffee Shop,Pizza Place,Italian Restaurant,Café,Bar,Sushi Restaurant,Pub,Park,Steakhouse,American Restaurant
43,M4M,East Toronto,Studio District,43.6628,-79.3348,2,Coffee Shop,Pizza Place,Italian Restaurant,Café,Bar,Sushi Restaurant,Pub,Park,Steakhouse,American Restaurant
44,M4N,Central Toronto,Lawrence Park,43.7282,-79.3871,0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Dessert Shop,Italian Restaurant,Restaurant,Gym,Health & Beauty Service


In [53]:


map_clusters = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Borough'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters